In [2]:
import IsingLib
import pyqubo
import numpy as np

city = 5
vartype = 'BINARY'
x = pyqubo.Array.create('x', (city, city), vartype)
tsp_model = IsingLib.Model(vartype)
dis_li = [[0, 10, 8, 8, 4], [10, 0, 6, 11, 7], [8, 6, 0, 14, 8], [9, 11, 14, 0, 6], [4, 7, 8, 6, 0]]
distance = np.zeros((city**2, city**2))
for i in range(city):
    point = (i + 1) % city
    distance[i*city: (i+1)*city, point*city: (point+1)*city] = dis_li
time_const = pyqubo.Sum(0, city, lambda i: (pyqubo.Sum(0, city, lambda j: x[i, j]) - 1) ** 2)
city_const = pyqubo.Sum(0, city, lambda j: (pyqubo.Sum(0, city, lambda i: x[i, j]) - 1) ** 2)
tsp_model.add_terms(H_0=distance, H_1=time_const, H_2=city_const)

In [3]:
np.set_printoptions(edgeitems=7)
A = 8
tsp_model.compile(H_1=A, H_2=A)
print(tsp_model.matrix)

[[-16.  16.  16.  16.  16.  16.  10.   8.   8.   4.  16.   0.   0.   0.
    0.  16.   0.   0.   0.   0.  16.   0.   0.   0.   0.]
 [  0. -16.  16.  16.  16.  10.  16.   6.  11.   7.   0.  16.   0.   0.
    0.   0.  16.   0.   0.   0.   0.  16.   0.   0.   0.]
 [  0.   0. -16.  16.  16.   8.   6.  16.  14.   8.   0.   0.  16.   0.
    0.   0.   0.  16.   0.   0.   0.   0.  16.   0.   0.]
 [  0.   0.   0. -16.  16.   9.  11.  14.  16.   6.   0.   0.   0.  16.
    0.   0.   0.   0.  16.   0.   0.   0.   0.  16.   0.]
 [  0.   0.   0.   0. -16.   4.   7.   8.   6.  16.   0.   0.   0.   0.
   16.   0.   0.   0.   0.  16.   0.   0.   0.   0.  16.]
 [  0.   0.   0.   0.   0. -16.  16.  16.  16.  16.  16.  10.   8.   8.
    4.  16.   0.   0.   0.   0.  16.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. -16.  16.  16.  16.  10.  16.   6.  11.
    7.   0.  16.   0.   0.   0.   0.  16.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0. -16.  16.  16.   8.   6.  16.  14.
    8.   0.   0.  

In [4]:
state, eng = tsp_model.solve(temperature_start=100000, temperature_end=0.001)
print('')
print(state, )
print('')
ans = state.reshape(city, city)
print(ans)
tsp_


[0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0]

[[0 0 0 1 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 1 0 0]]


In [5]:
import itertools

for v in itertools.permutations(range(city)):
    d = 0
    for i in range(city):
        d += dis_li[v[i-1]][v[i]]
    print(d, v)

40 (0, 1, 2, 3, 4)
39 (0, 1, 2, 4, 3)
47 (0, 1, 3, 2, 4)
43 (0, 1, 3, 4, 2)
48 (0, 1, 4, 2, 3)
45 (0, 1, 4, 3, 2)
35 (0, 2, 1, 3, 4)
36 (0, 2, 1, 4, 3)
44 (0, 2, 3, 1, 4)
45 (0, 2, 3, 4, 1)
43 (0, 2, 4, 1, 3)
43 (0, 2, 4, 3, 1)
37 (0, 3, 1, 2, 4)
42 (0, 3, 1, 4, 2)
39 (0, 3, 2, 1, 4)
47 (0, 3, 2, 4, 1)
35 (0, 3, 4, 1, 2)
38 (0, 3, 4, 2, 1)
40 (0, 4, 1, 2, 3)
44 (0, 4, 1, 3, 2)
38 (0, 4, 2, 1, 3)
47 (0, 4, 2, 3, 1)
35 (0, 4, 3, 1, 2)
40 (0, 4, 3, 2, 1)
45 (1, 0, 2, 3, 4)
43 (1, 0, 2, 4, 3)
47 (1, 0, 3, 2, 4)
38 (1, 0, 3, 4, 2)
47 (1, 0, 4, 2, 3)
40 (1, 0, 4, 3, 2)
35 (1, 2, 0, 3, 4)
35 (1, 2, 0, 4, 3)
40 (1, 2, 3, 0, 4)
40 (1, 2, 3, 4, 0)
37 (1, 2, 4, 0, 3)
39 (1, 2, 4, 3, 0)
43 (1, 3, 0, 2, 4)
38 (1, 3, 0, 4, 2)
44 (1, 3, 2, 0, 4)
47 (1, 3, 2, 4, 0)
35 (1, 3, 4, 0, 2)
43 (1, 3, 4, 2, 0)
44 (1, 4, 0, 2, 3)
39 (1, 4, 0, 3, 2)
42 (1, 4, 2, 0, 3)
48 (1, 4, 2, 3, 0)
36 (1, 4, 3, 0, 2)
45 (1, 4, 3, 2, 0)
43 (2, 0, 1, 3, 4)
45 (2, 0, 1, 4, 3)
42 (2, 0, 3, 1, 4)
35 (2, 0, 3, 4, 1)
44 (2, 0, 4,

In [6]:
mat, const = tsp_model.get_params('da')
da = IsingLib.DaFormat(mat, api_key='/Users/daisuke.oku/da_key_2.json', version=2)
job_id = da.da_solver()
da.get_job(job_id)

In [7]:
da.get_energy(job_id)
state_li = da.get_configuration(job_id)
for state in state_li:
    np_state = np.array(state).reshape(city, city)
    print(np_state, tsp_model.energy(state))

[[0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]] -56.0
[[0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]] -56.0


In [23]:
import dimod